# Modifying Metadata

In this example we will show how to modify the metadata of a SDFITS file using `dysh`.

You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/metadata_management.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/metadata_management.ipynb" download>here</a> and selecting "Save Link As".

## Background

We will use a practical example. For observations with the GBT it is recommended to observe a flux density calibrator (see e.g., [Perley & Butler 2017](https://ui.adsabs.harvard.edu/abs/2017ApJS..230....7P/abstract) for a list of calibrator sources and their flux densities) with the same configuration as that used for the science observations. The reason being that the values of the temperature equivalent power of the noise diodes stored in the SDFITS files can be out of date.

In this example we won't use observations of a flux density calibrator, but instead we will use the analysis of [Goddy et al (2020)](https://ui.adsabs.harvard.edu/abs/2020RNAAS...4....3G/abstract). They find that the temperature stored in the SDFITS files is on average lower than the measured values, so that the temperature must be corrected by 20%
$$
T_{\rm{CAL,corr}}=1.2T_{\rm{CAL,file}}.
$$
Here we use this to correct the temperature of the noise diode.

## Loading Modules
We start by loading the modules we will use for this example. 

For display purposes, we use the static (non-interactive) matplotlib backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on jupyter lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for working with the data.
from dysh.fits.gbtfitsload import GBTFITSLoad

# These modules are only used to download the data.
from pathlib import Path
from dysh.util.download import from_url

### Data retrieval

We download the data we will use for this example, if necessary.

In [ ]:
url = "https://www.gb.nrao.edu/dysh/example_data/positionswitch/data/AGBT05B_047_01/AGBT05B_047_01.raw.acs/AGBT05B_047_01.raw.acs.fits"
savepath = Path.cwd() / "data"
savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
filename = from_url(url, savepath)

### Data loading

We load the data and inspect its contents.

In [ ]:
sdfits = GBTFITSLoad(filename)

In [ ]:
sdfits.summary()

### Metadata inspection

Now we inspect the current noise diode temperature stored in the SDFITS file.

In [ ]:
sdfits["TCAL", "PLNUM"]

For polarization 0 the noise diode temperature is 1.452650 K and for polarization 1 it is 1.424292 K.

We will calibrate the data using these values to compare after we update the noise diode temperature. We use position switching calibration, then we time average all the scans and remove an order 1 polynomial.

In [ ]:
ps_original = sdfits.getps(plnum=0, ifnum=0, fdnum=0).timeaverage()
ps_original.baseline(degree=1, remove=True)

### Metadata update

Now we update the temperature of the noise diode by multiplying by 1.2.

In [ ]:
sdfits["TCAL"] *= 1.2

Now we check that the values were updated.

In [ ]:
sdfits["TCAL", "PLNUM"]

The values were updated. We proceed with the data reduction.

In [ ]:
ps_updated = sdfits.getps(plnum=0, ifnum=0, fdnum=0).timeaverage()
ps_updated.baseline(degree=1, remove=True)

Now plot and compare the result. Since the antenna temperature is directly proportional to the temperature of the noise diode, now the line profile after the update should be 20% brighter than without the update.

In [ ]:
ps_original.plot(ymin=-0.5, ymax=0.3)
ps_updated.plot(ymin=-0.5, ymax=0.3)